# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now the CSV file titled <font color=red>event_datafile_new.csv</font> is ready for work, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

#### Create Keyspace

In [6]:
create_keyspace="""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
try:
    session.execute(create_keyspace)
except Exception as e:
    print(f"create keyspace failed !! Error : {e}")

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### There is a need to create tables to run the following queries, because with Apache Cassandra  the database tables are modelled on the queries that will be run subsequently.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


---------------------------------------------------------------------------------------------------------------------------------

## Query 1 

### Since a query will be run on sessionId and itemInSession, our primary key must have these columns which can then be partition on  the data on sessionId.

### Our Select query : SELECT artist, song, length FROM  session_item where sessionId = 338 and itemInSession = 4
### Our Primary key will be (sessionId, itemInSession), where sessionId is the partition key and  itemInSession is the clustering column.
### Columns we included in the table : 

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_library"
query= query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
    print("Table Created Successfully")
except Exception as e:
    print(f"failed to create table!! Error : {e}")
                    

Table Created Successfully


#### Selecting the required columns from the csv file into the Apache Cassansa 

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

#### A SELECT query to verify that the data have been inserted into each table

In [11]:
select_query1 = "SELECT artist, song, length FROM  session_library where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2

### Since a query will be run on sessionId and userId, which will be sorted on itemInSession. So, the primary key must have these columns, which  can be partition on a composite key (sessionId, userId).

### Our Select query : SELECT artist, song, firstName, lastName FROM  userid_session where sessionId = 182 and userId = 10
### Our Primary key will be ((sessionId, userId), itemInSession)), where (sessionId, userId) is the partition key and  itemInSession is the clustering column.
### Also, we are using the clause - WITH CLUSTERING ORDER BY (itemInSession ASC), to sort our data based on itemInSession
### Columns we included in the table : sessionId, userId, itemInSession, artist, song, firstName, lastName

In [12]:
query2= "CREATE TABLE IF NOT EXISTS userid_session"
query2= query2 + "(sessionId int, userId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) "
try:
    session.execute(query2)
except Exception as e:
    print(f"Failed to create table! : {e}")

                    

#### A SELECT query to verify that the data have been inserted into each table

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO userid_session (sessionId, userId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]),int(line[3]), line[0],line[9],line[1],line[4]))

#### A SELECT query to verify that the data have been inserted into each table

In [14]:
select2 = "SELECT artist, song, firstName, lastName FROM  userid_session where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3

### Since we will need to run query on song our primary key must have song. Also, the query should be such that it does not contain duplicate users for a song the userId will be included in the primary key.

### Our Select query : SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'
### Our Primary key will be ((song), userId)), where song is the partition key and  userId is the clustering column.

In [15]:
query3 = "CREATE TABLE IF NOT EXISTS user_song" 
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"
try: 
    session.execute(query3)
    print("Created Table Successfully!!")
except Exception as e:
    print(f"Failed to Create Table!! Error : {e}")

Created Table Successfully!!


#### A SELECT query to verify that the data have been inserted into each table

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (  line[9], int(line[10]), line[1], line[4] )  )

### Drop the tables before closing out the sessions

In [22]:
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")
session.execute("DROP TABLE IF EXISTS sparkify.user_song")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()